# Imports & Config

In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
import datetime 
import random
from random import randrange
import os 
import faker
from faker import Faker
from random import randint
from dateutil.relativedelta import *
import copy
import importlib

In [ ]:
#connect BQ
from google.cloud import bigquery

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/leighajarett/Documents/service accounts/lookerdata-e7f3d750308e.json'

bq_client = bigquery.Client()

In [ ]:
#connect Looker
from looker_sdk import client, models, error
from run_look_with_filters import run_look_with_filters

#sdk = client.setup("/Users/leighajarett/looker.ini")
sdk = client.setup("/Users/leighajarett/looker_pebl.ini")

# Changing Districts

In [ ]:
new_districts = pd.read_excel('/Users/leighajarett/Documents/retail_banking/district_to_zip.xlsx')

In [ ]:
#new_districts['city'].unique()

In [ ]:
# query_to_run = 'select *, '
# end = 'FROM `lookerdata.retail_banking.district`'
# for i in range(len(new_districts)):
#     if i == 0:
#         new_string = 'case when district_id = ' 
#     else:
#         new_string = 'when district_id = '
#     new_string += str(new_districts['district_code'][i])
#     new_string += ' then %s' % ('"' + str(new_districts['zipcode'][i]) + '"')
#     print(new_string)

# Changing the Credit Card Table

Import Original Cards Table From BQ - base data comes from here https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions

In [ ]:
sql = 'SELECT * FROM `lookerdata.retail_banking.card` '
cards = bq_client.query(sql).to_dataframe()

In [ ]:
Counter(cards['type'])

We want to create two new card types: junior2 and classic2, people who joined each month and had < 60000, or >40000, and had either Junior or Classic, as the latest balance 

In [ ]:
q = sdk.look(22).query
filters = q.filters

In [ ]:
for card_type in (['junior2', 'classic2']):
    total_number = 0
    number_changed = 0
    filters = {}
    
    if card_type == 'Junior2':
        filters['card.type'] = 'Classic,Junior'
        filters['account_latest_balance.latest_balance'] = '<60000'
    else:
        filters['card.type'] = 'Classic'
        filters['account_latest_balance.latest_balance'] = '>40000'

    
    for month in range(1,13):
        for year in range(1993,1997):
            print(month,'-',year)
            mt = str(month)
            yr = str(year)
            if len(mt) > 1:
                filters['account.create_month'] = '%s-%s' %(yr,mt)
            else:
                filters['account.create_month'] = '%s-0%s' %(yr,mt)
            results = run_look_with_filters(22,filters,sdk)
            total_number += len(results)
            if len(results) > 2:
                #figure out how many cards we want to convert
                print(int(round(.7*len(results))))
                n = randrange(int(round(.7*len(results)))) # up to 70% of the results
                number_changed += n
                for i in range(n):
                    #print(results)
                    card_id = results[randrange(len(results))]['card.card_id'] #random index 
                    cards.loc[cards['card_id'] == card_id,'type'] = card_type
    
        print('Total number for ', card_type, ': ', total_number)
        print('Number changed for ', card_type, ': ', number_changed)

In [ ]:
Counter(cards['type'])

We also want to make end dates for some cards, and introduce new cards so they can have more than one at one time 
or convert to other cards.

Introduce a few scenarios...

1. 90% of people who are in the age group 18-25 and have a Junior1
        --> make it so they switched to either classic or Junior1 after ~1 years

In [ ]:
#initialize end date column
cards['end_date'] = np.nan
#cards['date'] = cards['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d %H:%M:%S')) 
#cards['date'] = cards['date'].apply(lambda x: pd.Timestamp(x))
#cards['date'] = cards['date'].dt.to_pydatetime() 

In [ ]:
filters['account.create_month'] = ''
filters['card.type'] = 'Junior'
filters['account_latest_balance.latest_balance'] = ''
filters['client.age_tier'] = '18 to 24'

results = run_look_with_filters(22,filters,sdk)
results = pd.DataFrame(results)

#randomly choose 90% of card holders
results = results.sample(frac=1)[0:round(len(results)*.9)].reset_index()

#randomly generate a normal distribution of time they convert to next card, and the time they take to cancel previous
days_to_add = list(np.random.normal(365, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))

#70% will go to Junior1 and 30% will go to classic1
new_card_id = max(cards['card_id'])
new_cards = []

for i in range(len(results)):
    card_id = results.loc[i,'card.card_id']
    disp_id = cards.loc[cards['card_id'] == card_id,'disp_id'].values[0]

    new_card_id += 1
    new_card_date = pd.to_datetime(cards.loc[cards['card_id'] == card_id,'date'
                             ].values[0]) + datetime.timedelta(days=days_to_add.pop()) 
    
    #fix dates that were created after they should have been
    if pd.to_datetime(new_card_date) > datetime.datetime(1998,12,31,0,0):
        delta = datetime.datetime(1998,12,31,0,0) - datetime.datetime(1997,12,31,0,0)
        int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
        new_card_date = datetime.datetime(1997,12,31,0,0) + datetime.timedelta(seconds=randrange(int_delta))
    
    cancel_date = new_card_date +datetime.timedelta(days=days_to_cancel.pop()) 
    
    if pd.to_datetime(cancel_date) > datetime.datetime(1998,12,31,0,0):
        cancel_date = np.nan
    
    if i < round(round(len(results)*.9)*.7):
        card_type = 'junior2'
    else:
        card_type = 'classic'
        
    #update end_date for existing
    cards.loc[cards['card_id'] == card_id,'end_date'] = cancel_date 
    
    #update cards
    new_card = dict()
    new_card['card_id'] = new_card_id
    new_card['disp_id'] = disp_id
    new_card['type'] = card_type
    new_card['date'] = new_card_date
    new_card['end_date'] = np.nan
    new_cards.append(new_card)

cards = cards.append(pd.DataFrame(new_cards))    

In [ ]:
Counter(cards['type'])

In [ ]:
Counter(cards[~cards['end_date'].isnull()]['type'])

2. 70% of People with gold card started with either Junior1, Classic1 or Classic2
        --> 20% of came from gold first
        --> 60% of came from Classic2
        --> 10% of came from Junior1
        --> 10% of came from Classic
        
        --> 10% of the above ^ canceled their first card sometime after joining

In [ ]:
filters['account.create_month'] = ''
filters['card.type'] = 'Gold'
filters['account_fact.latest_balance'] = ''
filters['client.age_tier'] = ''

results = run_look_with_filters(22,filters,sdk)
results = pd.DataFrame(results).sample(frac=1).reset_index()

ix_cl = round(len(results)*.6)
ix_j2 = ix_cl+round(len(results)*.1)
ix_cl2 = ix_j2+round(len(results)*.1)

rand_10 = random.sample(range(0, ix_cl2), round(ix_cl2*.25))

#for each...
#change the original card_type 
#then randomly chose a date between now and when the original card was created to sign up for gold
#also if index is in randomly chosen 10% then chose an end date for the gold card

new_cards = []
new_card_id = max(cards['card_id'])
curr_ix = 0
for ix in [ix_cl,ix_j2,ix_cl2]:
    for i in range(curr_ix,ix):
        card_id = results.loc[i,'card.card_id']
        disp_id = cards.loc[cards['card_id'] == card_id,'disp_id'].values[0]
        if ix == ix_cl:
            card_type = 'classic2'
        elif ix == ix_j2:
            card_type = 'junior2'
        else:
            card_type = 'classic'
        
        cards.loc[cards['card_id'] == card_id,'type'] = card_type

        new_card_id += 1
        
        created_date = pd.to_datetime(cards.loc[cards['card_id'] == card_id,'date'
                             ].values[0])
        
        if created_date > datetime.datetime(1998,12,31,0,0):
            delta = datetime.datetime(1998,12,31,0,0) - datetime.datetime(1997,12,31,0,0)
            int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
            created_date = datetime.datetime(1997,12,31,0,0) + datetime.timedelta(seconds=randrange(int_delta))
        
        new_card_date = created_date + datetime.timedelta(days=randrange((
            datetime.datetime(1998, 12, 31, 0, 0) - created_date).days)) 
        
        if new_card_date > datetime.datetime(1998,12,31,0,0):
            new_card_date = datetime.datetime(1998, 12, 31, 0, 0)
        
        new_cancel_date = np.nan
        
        if i in rand_10[0:15]:
            new_cancel_date = new_card_date + datetime.timedelta(days=randrange((
                datetime.datetime(1998, 12, 31, 0, 0) - new_card_date).days)) 
        
        elif i in rand_10[15:]:
            cards.loc[cards['card_id'] == card_id,'end_date'] = created_date + datetime.timedelta(days=randrange((
                datetime.datetime(1998, 12, 31, 0, 0) - created_date).days)) 
        
        
        #update cards
        new_card = dict()
        new_card['card_id'] = new_card_id
        new_card['disp_id'] = disp_id
        new_card['type'] = 'gold'
        new_card['date'] = new_card_date
        new_card['end_date'] = new_cancel_date
        new_cards.append(new_card)
        
    curr_ix = ix
cards = cards.append(pd.DataFrame(new_cards))   

In [ ]:
Counter(cards['type'])

In [ ]:
Counter(cards[~cards['end_date'].isnull()]['type'])

3. 70% People convert with classic2 came from classic

In [ ]:
results = cards.loc[cards['type']=='classic2'].reset_index()

#randomly generate a normal distribution of time they convert to next card, and the time they take to cancel previous
days_to_add = list(np.random.normal(365, 150, len(results)))
days_to_cancel = list(np.random.normal(30*3, 25, len(results)))

new_cards = []
new_card_id = max(cards['card_id'])

for i in range(0,round(len(results)*.6)):
    card_id = results.loc[i,'card_id']
    disp_id = cards.loc[cards['card_id'] == card_id,'disp_id'].values[0]
    new_card_id += 1
    
    created_date = pd.to_datetime(cards.loc[cards['card_id'] == card_id,'date'
                             ].values[0])
    
    if created_date > datetime.datetime(1998,12,31,0,0):
        delta = datetime.datetime(1998,12,31,0,0) - datetime.datetime(1997,12,31,0,0)
        int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
        created_date = datetime.datetime(1997,12,31,0,0) + datetime.timedelta(seconds=randrange(int_delta))
        
    
    new_card_date = created_date - datetime.timedelta(days=days_to_add.pop()) 
    
    if new_card_date > datetime.datetime(1998,12,31,0,0):
        delta = datetime.datetime(1998,12,31,0,0) - created_date
        new_card_date = created_date - datetime.timedelta(days=randrange(delta))
    
    cancel_date = created_date + datetime.timedelta(days=days_to_cancel.pop()) 
    
    if cancel_date > datetime.datetime(1998,12,31,0,0):
        delta = datetime.datetime(1998,12,31,0,0) - created_date
        cancel_date = created_date + datetime.timedelta(days=randrange(delta.days))
    
    
    cards.loc[cards['card_id'] == card_id,'end_date'] = np.nan 
    
    new_card = dict()
    new_card['card_id'] = new_card_id
    new_card['disp_id'] = disp_id
    new_card['type'] = 'classic'
    new_card['date'] = new_card_date
    new_card['end_date'] = cancel_date
    new_cards.append(new_card)

cards = cards.append(pd.DataFrame(new_cards))     

In [ ]:
Counter(cards['type'])

In [ ]:
Counter(cards[~cards['end_date'].isnull()]['type'])

In [ ]:
cards = cards.reset_index()
cards['end_date'] = cards['end_date'].apply(lambda x: pd.to_datetime(x))
cards.loc[cards['end_date'] < cards['date'],'end_date'] = cards.loc[cards['end_date'] < cards['date'],['date','end_date']].apply(
    lambda x:  x['date'] + datetime.timedelta(days = randrange((datetime.datetime(1998,12,31,0,0) - x['date']).days))
                                              ,axis=1)

Write the cards back to the database

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'
          ] = '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json'
bq_client = bigquery.Client()

In [ ]:
cards.to_gbq('retail_banking.base_card',if_exists = 'replace')

# sql = "create or replace table `looker-private-demo.retail_banking.base_card`  \
# as \
# SELECT * except(date, end_date) \
#   , CAST(PARSE_DATETIME('%Y-%m-%d', date) as date) as date \
#   , CAST(PARSE_DATETIME('%Y-%m-%d', end_date) as date) as end_date \
#   FROM `looker-private-demo.retail_banking.base_card`"
# print(bq_client.query(sql).to_dataframe())

# Adding to the Clients Table 

In [ ]:
#import os
os.chdir("/Users/leighajarett/Sparkov_Data_Generation/")
import datagen_customer
importlib.reload(datagen_customer)
from datagen_customer import Headers, Customer
#from demographics import make_cities, make_age_gender_dict
import main_config
importlib.reload(main_config)
from main_config import MainConfig
#os.chdir("/Users/leighajarett/")

In [ ]:
# seed_num = 4444
# cities = make_cities()
# age_gender = make_age_gender_dict()
# fake = Faker()
# Faker.seed(seed_num)

In [ ]:
import run_look_with_filters
importlib.reload(run_look_with_filters)
from run_look_with_filters import run_look_with_filters
from looker_sdk import client, models, error

Get original clients table

In [ ]:
os.chdir("/Users/leighajarett/")
sdk = client.setup("/Users/leighajarett/looker.ini")

q = sdk.look(410).query
filters = q.filters

clients_ = run_look_with_filters(410,filters,sdk)
clients_ = pd.DataFrame(clients_)

os.chdir("/Users/leighajarett/Sparkov_Data_Generation/")

For each client, go through and create a Customer object, which will randomly generate other attributes

In [ ]:
headers = Headers()
headers.save_headers_to_file('clients_data.csv')

In [ ]:
fake = Faker()
Faker.seed(444)

client_array = []
for i in range(len(clients_)):
    client_ = clients_.iloc[i]
    if client_['district.city'] == 'Longmount':
        cust = Customer(fake, client_['client.gender'],client_['client.birth_date'], 'Longmont')
    elif client_['district.city'] == 'Thornton':
        cust = Customer(fake, client_['client.gender'],client_['client.birth_date'], 'Denver')
    else:
        cust = Customer(fake, client_['client.gender'],client_['client.birth_date'], client_['district.city'])
    c = cust.print_customer()
    c['client_id'] = client_['client.client_id']
    c['district_id'] = client_['client.district_id']
    c['card_id'] = client_['card.card_id']
    c['disp_id'] = client_['disp.disp_id']
    cust.save_to_file('clients_data.csv')
    
    client_array.append(c)

Make a mapping of cc_number to card_id

In [ ]:
new_client_df = pd.DataFrame(client_array)

In [ ]:
Counter(new_client_df['profile'])

Make sure to take out the duplicate rows for client id after we match credit card numbers

In [ ]:
card_lookup = dict(zip(new_client_df['card_id'],new_client_df['cc_number']))
rev_card_lookup = dict(zip(new_client_df['cc_number'],new_client_df['card_id']))
new_client_df = new_client_df.loc[:,['ssn','first_name','last_name','gender','street','address',
              'job','dob','profile','client_id','district_id','disp_id']]
new_client_df = new_client_df.drop_duplicates('client_id','first')

Add card numbers to table

In [ ]:
#query_to_run = 'create or replace table `retail_banking.base_card` as (select * except (card_number), '
query_to_run = 'create or replace table `retail_banking.base_card` as (select *, '
end = ' FROM `retail_banking.base_card` )'
for i in range(len(card_lookup.keys())):
    if i == 0:
        new_string = 'case when card_id = ' 
    else:
        new_string = ' when card_id = '
    new_string += str(list(card_lookup.keys())[i])
    new_string += ' then %s' % ('"' + str(card_lookup[list(card_lookup.keys())[i]]) + '"')
    query_to_run += new_string 
query_to_run += ' else null end as card_number' + end

In [ ]:
q_result = bq_client.query(query_to_run)
q_result.to_dataframe()

Replace client table

In [ ]:
def create_traffice_source(profile):
    traffic_sources = ['Mailer','Email','Call Center','Website','Third Party Advertisement']
    if 'young_adults' in profile:
        #more likely to be third party adverisement or website
        traffic_sources = traffic_sources + [traffic_sources[4],traffic_sources[4],traffic_sources[4
                                    ],traffic_sources[3],traffic_sources[3]]
    elif 'urban' in profile:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[2],traffic_sources[2
                        ],traffic_sources[3],traffic_sources[3]]
    else:
        traffic_sources = traffic_sources + [traffic_sources[1],traffic_sources[1],traffic_sources[1
                ],traffic_sources[2],traffic_sources[3]]
    
    return np.random.choice(traffic_sources)

In [ ]:
new_client_df['traffice_source'] = new_client_df['profile'].apply(lambda x: create_traffice_source(x))
new_client_df['profile'] = new_client_df['profile'].apply(lambda x: x[0:-5])

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'
          ] = '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json'
bq_client = bigquery.Client()

In [ ]:
new_client_df.to_gbq('retail_banking.client',if_exists = 'replace')

In [ ]:
#make sure to go update the tables in looker-private-demo from here

# Making the Transactions Table

Use

In [ ]:
from os import listdir
from os.path import isfile, join
from datetime import date

In [ ]:
profile_files = [f for f in listdir('/Users/leighajarett/Sparkov_Data_Generation/profiles/'
                                   ) if isfile(join('/Users/leighajarett/Sparkov_Data_Generation/profiles/', f))]
profile_files = [join('/Users/leighajarett/Sparkov_Data_Generation/profiles/', f) 
                 for f in profile_files if f.split('_')[0] in ('adults','young')]

In [ ]:
import profile_weights
from datagen_transaction import T_Customer, create_header

In [ ]:
import datagen_transaction
importlib.reload(datagen_transaction)
from datagen_transaction import T_Customer, create_header


importlib.reload(profile_weights)

In [ ]:
customers_file = open('clients_data.csv', 'r').readlines() 

In [ ]:
headers = create_header(customers_file[0])

In [ ]:
left_out_cards = pd.read_csv('/Users/leighajarett/Documents/retail_banking/missing_card_numbers.csv',header=None)

In [ ]:
for i in left_out_cards[1].astype(int).astype(str):
    print(i)

In [ ]:
len(customers_file)

In [ ]:
len(cards)

In [ ]:
count = 0
for i in customers_file:
    #print(i.split('|')[1])
    if i.split('|')[1] in list(left_out_cards[1].astype(int).astype(str)):
        count+=1
        print(count)

In [ ]:
txns = []
cards_w_txns = []
counter = 0
len_txns = 0
for cust in customers_file[1:]:
    if cust.split('|')[1] in list(left_out_cards[1].astype(int).astype(str)):
        for m in profile_files:
        #for m in ['/Users/leighajarett/Sparkov_Data_Generation/profiles/young_adults_male_rural.json']:
            pro_name = m.split('profiles')[-1]
            pro_name = pro_name[1:]
            parse_index = m.index('profiles') + 9
            m_fraud = m[:parse_index] +'fraud_' + m[parse_index:]
            pro = open(m, 'r').read()
            pro_fraud = open(m_fraud, 'r').read()
            pro_name_fraud = 'fraud_' + pro_name

            curr_profile = pro_name
            curr_fraud_profile = pro_name_fraud
            profile_name = m

            card_id = rev_card_lookup[cust.split('|')[1]]
            cards_w_txns.append(card_id)
            create_d = clients_[clients_['card.card_id'] == card_id]['card.create_date']
            create_d = date(int(create_d.values[0].split('-')[0]),
                             int(create_d.values[0].split('-')[1]),
                             int(create_d.values[0].split('-')[2]))

            end_d = clients_[clients_['card.card_id'] == card_id]['card.end_date']
            if end_d.values[0] is None:
                end_d = date(2020,12,31)
                if (end_d-create_d).days < 10:
                    end_d = date(2021,1,1) + datetime.timedelta(days=10)
            else:
                end_d = date(int(end_d.values[0].split('-')[0]),
                     int(end_d.values[0].split('-')[1]),
                     int(end_d.values[0].split('-')[2]))
            if create_d > date(2020,12,31):
                create_d = date(2020,12,31)
                end_d = date(2021,1,1) + datetime.timedelta(days=10)

            profile = profile_weights.Profile(pro,create_d,end_d)
            t_cust = T_Customer(cust, profile, headers, fake)

            if t_cust.attrs['profile'] == curr_profile:
                merch = pd.read_csv('data/merchants.csv', sep='|')
                is_fraud= 0
                fraud_flag = randint(0,100) 
                fraud_dates = []

                # decide if we generate fraud or not
                if fraud_flag < 30: #11->25
                    fraud_interval = randint(1,1) #7->1
                    inter_val = (end_d-create_d).days-7
                    # rand_interval is the random no of days to be added to start date
                    if inter_val <= 0:
                        inter_val = 1
                    rand_interval = randint(1, inter_val)
                    #random start date is selected
                    newstart = create_d + datetime.timedelta(days=rand_interval)
                    # based on the fraud interval , random enddate is selected
                    newend = newstart + datetime.timedelta(days=fraud_interval)
                    # we assume that the fraud window can be between 1 to 7 days #7->1
                    profile = profile_weights.Profile(pro_fraud, newstart, newend)
                    t_cust = T_Customer(cust, profile, headers, fake)
                    merch = pd.read_csv('data/merchants.csv' , sep='|')
                    is_fraud = 1
                    temp_tx_data = profile.sample_from(is_fraud)
                    fraud_dates = temp_tx_data[3]
                    txns = txns + t_cust.print_trans(temp_tx_data,is_fraud, fraud_dates,merch)
                    #parse_index = m.index('profiles/') + 9
                    #m = m[:parse_index] +'fraud_' + m[parse_index:]

                # we're done with fraud (or didn't do it) but still need regular transactions
                # we pass through our previously selected fraud dates (if any) to filter them
                # out of regular transactions
                profile = profile_weights.Profile(pro, create_d, end_d)
                merch = pd.read_csv('data/merchants.csv', sep='|')
                is_fraud =0
                temp_tx_data = profile.sample_from(is_fraud)
                txns = txns + t_cust.print_trans(temp_tx_data, is_fraud, fraud_dates, merch)
#                 len_txns += len(txns)

#                 if counter == 0: 
#                     pd.DataFrame(txns).to_gbq('retail_banking.test_trans',if_exists = 'replace')
#                 else:
#                     pd.DataFrame(txns).to_gbq('retail_banking.test_trans',if_exists = 'append')
#                 print('Added ', len(txns),' to the table, for card ', card_id)
                counter += 1
                print(counter)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'
          ] = '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json'
bq_client = bigquery.Client()

In [ ]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json')

In [ ]:
pd.DataFrame(txns).to_gbq('retail_banking.card_transactions',if_exists = 'append',credentials = credentials)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'
          ] = '/Users/leighajarett/Documents/service accounts/lookerdata-e7f3d750308e.json'
bq_client = bigquery.Client()

# Credit Card Payments

Payments:
    50% of people will be carrying a balance
    5% of people will have defaulted on a credit card
    45% of people pay their credit card off on time every month

Randomly choose day of month their payments are due for each card

In [ ]:
#Calculate the balance for each day, for each card....

sql = "SELECT card.card_id  AS card_card_id, \
        TIMESTAMP(CONCAT(card_transactions.trans_date,' ',card_transactions.trans_time)) \
                AS card_transactions_transaction_date, \
        cast(card_transactions.amount as FLOAT64)  AS card_transactions_amount,\
        sum(cast(card_transactions.amount as FLOAT64)) OVER \
                (PARTITION BY card.card_id, date \
                ORDER BY TIMESTAMP(CONCAT(card_transactions.trans_date,' ',card_transactions.trans_time))asc) \
FROM `looker-private-demo`.retail_banking.card_transactions  AS card_transactions \
LEFT JOIN retail_banking.card  AS card ON card.card_number = card_transactions.cc_number \
where card.card_id is not null \
order by card.card_id , TIMESTAMP(CONCAT(card_transactions.trans_date,' ',card_transactions.trans_time)) "


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'
          ] = '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json'
bq_client = bigquery.Client()

In [ ]:
transactions = bq_client.query(sql).to_dataframe()
sql = 'SELECT * FROM `looker-private-demo.retail_banking.card` '
cards = bq_client.query(sql).to_dataframe()

In [ ]:
cards['payment_day_of_month'] = cards['card_id'].apply(lambda x: randrange(1,28))

In [ ]:
# cards.to_gbq('retail_banking.base_card',if_exists = 'replace')

# sql = "create or replace table `looker-private-demo.retail_banking.base_card`  \
# as \
# SELECT * except(date, end_date) \
#   , CAST(PARSE_DATETIME('%Y-%m-%d', date) as date) as date \
#   , CAST(PARSE_DATETIME('%Y-%m-%d', end_date) as date) as end_date \
#   FROM `looker-private-demo.retail_banking.base_card`"
# bq_client.query(sql)

In [ ]:
#pay balance completely (no penalty) 0 
#pay balance partially (interest + maybe late fee) 1 
#do not pay balance at all (interest + maybe late fee) 2 

#keep record of past 3 months...
    # * * 0 --> if they have not missed a payment last month there is only 10% chance 2, 10% chance of 1
    # 2 2 2 --> if they missed their payment the past 3 months there is a 90% chance they miss it now
    # * * 1/2 --> if they paid partially last month, and missed or paid partially before, 20% chance of 1 20% chance of 2


In [ ]:
#Figure out the dates payments are due for each card

payment_dates = []
cards['end_date'] = pd.to_datetime(cards['end_date'])

days_before_payment = [19,18,17,16,15,13,12,11,10,9,8,7,6,
 5,5,5,
 4,4,4,
 3,3,3,3,3,3,3,
 2,2,2,2,2,2,2,2,2,
 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

interest_and_latefees = {}

card_ids = list(cards['card_id'])
random.shuffle(card_ids)
full_payers = card_ids[0:int(len(card_ids)/2)]

last_card = 0
for c_id in cards['card_id'].unique():
    start_date = pd.to_datetime(cards.loc[cards['card_id'] == c_id,'date'].values[0])

    end_date = pd.to_datetime(cards.loc[cards['card_id'] == c_id,'end_date'].values[0])
    if pd.isna(end_date):
        end_date = date(2020,12,31)
    else:
        ts = (end_date - np.datetime64('1970-01-01')) / np.timedelta64(1, 's') 
        end_date = datetime.datetime.utcfromtimestamp(ts)
    
    payment_day = cards.loc[cards['card_id'] == c_id,'payment_day_of_month'].values[0]

    #if the start day is before the payment day, then payment period starts previous month
    if start_date.day < payment_day:
        period_start = date(start_date.year, start_date.month, payment_day) - relativedelta(months=+1)
    else:
        period_start = date(start_date.year, start_date.month, payment_day)


    #if the end day is after the payment day, then payment period starts the previous
    if end_date.day > payment_day:
        last_period_start = date(end_date.year, end_date.month, payment_day) 
    else:
        last_period_start = date(end_date.year, end_date.month, payment_day) - relativedelta(months=+1)
    
    while period_start <= last_period_start:
        if last_card != c_id:
            last_3_payments = [0,0,0]
        
        payment_date = {}
        #payment_date['card_number'] = card_lookup[c_id]
        payment_date['card_number'] = cards.loc[cards['card_id'] == c_id,'card_number'].values[0]
        payment_date['card_id'] = c_id
        payment_date['period_start'] = period_start
        period_end = period_start + relativedelta(months=+1)
        payment_date['period_end'] = period_end
        payment_date['payment_due_date'] = period_end + relativedelta(months=+1)
        
        #check to see if this person pays in full each month 
        if c_id in full_payers:
            payment_date['is_full_payer'] = 1
            #sometimes people make more than one payment a month
            if random.choice([1,2,3,4,5]) == 1 and last_3_payments[2] ==0:
                #print(c_id, period_start)
                other_payment = copy.deepcopy(payment_date)
                other_payment['payment_type'] = 1
                other_payment['balance_percent'] = random.gauss(50,15)
                other_payment['payment_completed_date'] = period_end + relativedelta(months=+1) -datetime.timedelta(
                    days=random.choice(list(range(20,28))))
                payment_dates.append(other_payment)
                last_3_payments.pop(0)
                last_3_payments.append(1)
                
            payment_date['payment_type'] = 0
            payment_date['balance_percent'] = 100
            payment_date['payment_completed_date'] = period_end + relativedelta(months=+1) -datetime.timedelta(
                days=random.choice(days_before_payment))
            payment_dates.append(payment_date)
            last_3_payments.pop(0)
            last_3_payments.append(0)
        
        else:
            payment_date['is_full_payer'] = 0
            rand_int = random.randint(1,100)
            if last_3_payments[2] == 0:
                #if they have not missed a payment last month there is only 20 % chance they default or pay partially
                if rand_int <= 10: #pay partially
                    payment_date['payment_type'] = 1
                    payment_date['balance_percent'] = random.gauss(50,15)
                    payment_date['payment_completed_date'] = period_end + relativedelta(
                        months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                    payment_dates.append(payment_date)
                    last_3_payments.pop(0)
                    last_3_payments.append(1)
                    
                elif rand_int > 20: #pay balance in full
                    payment_date['payment_type'] = 0
                    payment_date['balance_percent'] = 100
                    payment_date['payment_completed_date'] = period_end + relativedelta(
                        months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                    payment_dates.append(payment_date)
                    last_3_payments.pop(0)
                    last_3_payments.append(0)
                
                else: #default
                    payment_date['payment_type'] = np.nan
                    payment_date['balance_percent'] = 0
                    payment_date['payment_completed_date'] = np.nan
                    payment_dates.append(payment_date)
                    last_3_payments.pop(0)
                    last_3_payments.append(2)
            
            else:
                if random.choice([1,2,3,4,5]) == 1:
                    other_payment = copy.deepcopy(payment_date)
                    other_payment['payment_type'] = 1
                    other_payment['balance_percent'] = random.gauss(50,15)
                    other_payment['payment_completed_date'] = period_end + relativedelta(months=+1
                                                    ) -datetime.timedelta(days=random.choice(list(range(20,28))))
                    payment_dates.append(other_payment)
                    last_3_payments.pop(0)
                    last_3_payments.append(1)
                if last_3_payments[0] == 2 and last_3_payments[1] == 2:
                    ### if they missed their payment the past 3 months there is a 80% chance they default
                    if rand_int <= 80:
                        payment_date['payment_type'] = np.nan
                        payment_date['balance_percent'] = 0
                        payment_date['payment_completed_date'] = np.nan
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(2)
                    elif rand_int <= 90:
                        payment_date['payment_type'] = 1
                        payment_date['balance_percent'] = random.gauss(50,15)
                        payment_date['payment_completed_date'] = period_end + relativedelta(
                            months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(1)
                    else:
                        payment_date['payment_type'] = 0
                        payment_date['balance_percent'] = 100
                        payment_date['payment_completed_date'] = period_end + relativedelta(
                            months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(0)
                
                else: 
                    if rand_int <= 20:
                        payment_date['payment_type'] = np.nan
                        payment_date['balance_percent'] = 0
                        payment_date['payment_completed_date'] = np.nan
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(2)
                    elif rand_int <= 40:
                        payment_date['payment_type'] = 1
                        payment_date['balance_percent'] = random.gauss(50,15)
                        payment_date['payment_completed_date'] = period_end + relativedelta(
                            months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(1)
                    else:
                        payment_date['payment_type'] = 0
                        payment_date['balance_percent'] = 100
                        payment_date['payment_completed_date'] = period_end + relativedelta(
                            months=+1) -datetime.timedelta(days=random.choice(days_before_payment))
                        payment_dates.append(payment_date)
                        last_3_payments.pop(0)
                        last_3_payments.append(0)
        
        last_card = c_id
        period_start = period_start + relativedelta(months=+1)

In [ ]:
payment_date

In [ ]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/Users/leighajarett/Documents/service accounts/looker-private-demo-d589d7060d61.json')

payment_dates = pd.DataFrame(payment_dates)
pd.DataFrame(payment_dates).to_gbq('finserv_staging.card_payment_dates',if_exists = 'replace',credentials = credentials)

In [ ]:
len(payment_dates)

In [ ]:
sql = "with trn_pmnt as (select * from  `looker-private-demo.retail_banking.card_transactions` as tns \
left join `looker-private-demo.finserv_staging.card_payment_dates` as pmnts \
on tns.cc_number = pmnts.card_number and timestamp(tns.trans_date) >= timestamp(pmnts.period_start) \
  and timestamp(tns.trans_date) < timestamp(pmnts.period_end) \
order by tns.cc_number, tns.trans_date) \
select * except (card_id, payment_due_date), due.card_id, due.payment_due_date \
from ( \
select card_id, payment_due_date, max(balance) as period_transactions_total from ( \
  select card_id, payment_due_date, trans_id, \
    TIMESTAMP(CONCAT(trn_pmnt.trans_date,' ',trn_pmnt.trans_time)) as trans_date, amount, \
  sum(cast(trn_pmnt.amount as FLOAT64)) OVER \
        (PARTITION BY trn_pmnt.card_id, payment_due_date \
        ORDER BY TIMESTAMP(CONCAT(trn_pmnt.trans_date,' ',trn_pmnt.trans_time))asc) as balance \
  from trn_pmnt ) as t \
  group by 1,2 \
) as due \
left join  `looker-private-demo`.finserv_staging.card_payment_dates as pmnts \
on due.card_id = pmnts.card_id and due.payment_due_date = pmnts.payment_due_date" 

In [ ]:
payments_w_amount = bq_client.query(sql).to_dataframe()

In [ ]:
payments_w_amount = payments_w_amount[~payments_w_amount['card_id'].isnull()].reset_index()

In [ ]:
last_card = 0
payments_w_amount['remaining_balance_last_period'] = 0
payments_w_amount['total_amount_owed'] = 0
payments_w_amount['total_amount_paid'] = 0
payments_w_amount['total_amount_paid'] = 0
payments_w_amount['total_interest_applied'] = 0

interest_rates = dict(zip(cards['type'].unique(),[.003, .005, .0035, .0025, .004]))

for i in range(len(payments_w_amount)):
    payment = payments_w_amount.loc[i]
    print(payment)
    
    if payment['card_id'] != last_card:
        remaining_balance_last_period = 0
        days_since_last_due = 0
    else:
        if payment['payment_completed_date'] is not None:
            completed_date = date(int(payment['payment_completed_date'].split('-')[0]),
                             int(payment['payment_completed_date'].split('-')[1]),
                             int(payment['payment_completed_date'].split('-')[2]))
        else:
            completed_date = date(int(payment['payment_due_date'].split('-')[0]),
                             int(payment['payment_due_date'].split('-')[1]),
                             int(payment['payment_due_date'].split('-')[2]))
            
        days_since_last_due = (completed_date - date(int(last_due.split('-')[0]),int(last_due.split('-')[1]),
                                        int(last_due.split('-')[2]))).days
        
    
    card_type = cards[cards['card_id'] == payment['card_id']]['type'].values[0]
    if days_since_last_due > 0:
        interest = remaining_balance_last_period*days_since_last_due*interest_rates[card_type]
    else: 
        interest = 0
  
    
    total_amount_owed =  remaining_balance_last_period + payment['period_transactions_total'] + interest
    total_amount_paid = total_amount_owed*(payment['balance_percent']/100)
    remaining_balance_this_period = total_amount_owed - total_amount_paid
    
    payments_w_amount.loc[i,'total_interest_applied'] = interest
    payments_w_amount.loc[i,'remaining_balance_last_period'] = remaining_balance_last_period
    payments_w_amount.loc[i,'total_amount_owed'] = total_amount_owed
    payments_w_amount.loc[i,'total_amount_paid'] = total_amount_paid
    payments_w_amount.loc[i,'remaining_balance_this_period'] = remaining_balance_this_period
    
    #print(interest, remaining_balance_last_period, total_amount_owed, total_amount_paid, remaining_balance_this_period)
    #print(days_since_last_due, last_due, completed_date)
    remaining_balance_last_period = remaining_balance_this_period
    last_due = payment['payment_due_date'] 
    last_card = payment['card_id']